<a href="https://colab.research.google.com/github/Hadiasemi/Data301/blob/main/Copy_of_3_7_Distance_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.7 Distance Metrics

So far, this chapter has been about ways to measure relationships between variables, or the _columns_ of a `DataFrame`. This lesson is about how to measure relationships between observations, or the _rows_ of a `DataFrame`.

How do we quantify how "similar" two observations are? We will use the Ames housing data set, but to keep things simple, we will work with just three quantitative variables from that data set: the number of bedrooms, the number of bathrooms, and the living area (in square feet).

In [16]:
import numpy as np
import pandas as pd

data_dir = "https://dlsun.github.io/pods/data/"
df_housing = pd.read_csv(data_dir + "AmesHousing.txt", sep="\t")

# extract 3 quantitative variables
df_housing_quant = df_housing[["Bedroom AbvGr", "Gr Liv Area"]].copy()
df_housing_quant["Bathrooms"] = (
    df_housing["Full Bath"] + 
    0.5 * df_housing["Half Bath"]
)
df_housing_quant

,Bedroom AbvGr,Gr Liv Area,Bathrooms
0,3,1656,1.0
1,2,896,1.0
2,3,1329,1.5
3,3,2110,2.5
4,3,1629,2.5
...,...,...,...
2925,3,1003,1.0
2926,2,902,1.0
2927,3,970,1.0
2928,2,1389,1.0


Shown below is a (three-dimensional) scatterplot of these variables. Consider the two observations connected by a red line. (The label next to each point is its index in the `DataFrame`.) To measure how similar they are, we can calculate the distance between the two points.

![](https://github.com/dlsun/pods/blob/master/03-Quantitative-Data/distance.png?raw=1)

Calculating the distance between two points is not as straightforward as it might seem because there is more than one way to define distance. The most familiar distance metric is probably _Euclidan distance_, which is the straight-line distance ("as the crow flies") between the two points. The formula for calculating this distance is a generalization of the Pythagorean theorem:

$$ d({\bf x}, {\bf x'}) = \sqrt{\sum_{j=1}^D (x_j - x'_j)^2} $$

In [17]:
x = df_housing_quant.loc[2927]
x1 = df_housing_quant.loc[2928]

x - x1

Bedroom AbvGr      1.0
Gr Liv Area     -419.0
Bathrooms          0.0
dtype: float64

In [18]:
(x - x1) ** 2

Bedroom AbvGr         1.0
Gr Liv Area      175561.0
Bathrooms             0.0
dtype: float64

In [19]:
np.sqrt(((x - x1) ** 2).sum())

419.0011933157231

The beauty of this definition is that it generalizes to more than three dimensions. Even though it is difficult to visualize points in 100-dimensional space, we can calculate distances between them in exactly the same way.

However, Euclidean distance is not the only way to measure how far apart two points are. There is also [**Manhattan distance**](https://en.wikipedia.org/wiki/Taxicab_geometry) (also called _taxicab distance_), which measures the distance a taxicab in Manhattan would have to drive to travel from A to B. In Manhattan, taxicabs cannot travel in a straight line (i.e., the green path below) because they have to follow the street grid. But there are multiple paths along the street grid that all have exactly the same length (i.e., the red, yellow, and blue paths below); the Manhattan distance is the length of any one of these shortest paths.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Manhattan_distance.svg/283px-Manhattan_distance.svg.png)

The formula for Manhattan distance is actually quite similar to the formula for Euclidean distance. Instead of squaring the differences and taking the square root at the end (as in Euclidean distance), we simply take absolute values:
$$ d({\bf x}, {\bf x'}) = \sum_{j=1}^D |x_j - x'_j|. $$

The following code calculates Manhattan distance:

In [20]:
((x - x1).abs()).sum()

420.0

In general, we can raise the absolute difference to any power $p$ and take the $p$th root. 
$$ d({\bf x}, {\bf x'}) = \left(\sum_{j=1}^D |x_j - x'_j|^p\right)^{1/p}. $$
This is called _Minkowski distance_. Manhattan distance and Euclidean distance are special cases of Minkowski distance for $p=1$ and $p=2$, respectively.

### Comparison of Euclidean and Manhattan distance

The Euclidean distance was essentially just the largest difference. This is because Euclidean distance first _squares_ the differences. The squaring operation has a "rich get richer" effect; larger values get magnified by more than smaller values. As a result, the largest differences tend to dominate the Euclidean distance.

On the other hand, Manhattan distance treats all differences equally. So Manhattan distance is preferred if we are concerned that an outlier in one variable might dominate the distance metric.

## The Importance of Scaling

Here's something to ponder. There are two pairs of observations in the figure below, one connected by a red line, the other connected by an orange line. Which pair of observations is more similar (assuming we use Euclidean distance)?

![](https://github.com/dlsun/pods/blob/master/03-Quantitative-Data/closer.png?raw=1)

Let's actually calculate these two distances.

In [21]:
# Distance between two points connected by red line
x = df_housing_quant.loc[2927]
x1 = df_housing_quant.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

419.0011933157231

In [22]:
# Distance between two points connected by orange line
x = df_housing_quant.loc[2498]
x1 = df_housing_quant.loc[290]

np.sqrt(((x - x1) ** 2).sum())


5.0990195135927845

Surprised by the answer? The scatterplot is deceiving because it automatically scales the variables to make the points fit on the same plot. In reality, the variables are on very different scales. The number of bedrooms and bathrooms range from 0 to 6, while living area is in the thousands. When variables are on such different scales, the variable with the largest variability will dominate the distance metric.

The plot below shows the same data, but drawn to scale. We can see that differences in the number of bedrooms and the number of bathrooms hardly matter at all; only the variability in the living area matters.

![](https://github.com/dlsun/pods/blob/master/03-Quantitative-Data/closer_rescaled.png?raw=1)

To obtain distances that agree more with our intuition---and that do not give too much weight to any one variable---we transform the variables to be on the same scale. There are several ways to _scale_ a variable ${\bf x} = (x_1, \ldots, x_n)$:

- _standardizing_: subtract each value by the mean, then divide by the standard deviation, 
$$ x_i \leftarrow \frac{x_i - \bar {\bf x}}{\text{SD}({\bf x})} $$
- _normalizing_: scale each value so that the variable has length (or "norm") 1, 
$$ x_i \leftarrow \frac{x_i}{\sqrt{\sum_{i=1}^n x_i^2}} $$
- _min/max scaling_: scale each value so that all values are between 0 and 1, 
$$x_i \leftarrow \frac{x_i - \min({\bf x})}{\max({\bf x}) - \min({\bf x})}.$$

The figure below illustrates what each of these scaling methods do to a synthetic data set with two variables. All three methods scale the variables in similar (but slightly different) ways, resulting in figure-eights with different aspect ratios.  Standardizing also moves the data to be centered around the origin, while min-max scaling moves the data to be in a box whose corners are $(0, 0)$ and $(1, 1)$.

![](https://github.com/dlsun/pods/blob/master/03-Quantitative-Data/scaling.png?raw=1)

Let's standardize the Ames housing data, and see how it affects the distance metric.

In [23]:
df_housing_st = (
    (df_housing_quant - df_housing_quant.mean()) / 
    df_housing_quant.std()
)
df_housing_st

,Bedroom AbvGr,Gr Liv Area,Bathrooms
0,0.176064,0.309212,-1.176462
1,-1.032058,-1.194223,-1.176462
2,0.176064,-0.337661,-0.398702
3,0.176064,1.207317,1.156819
4,0.176064,0.255801,1.156819
...,...,...,...
2925,0.176064,-0.982555,-1.176462
2926,-1.032058,-1.182354,-1.176462
2927,0.176064,-1.047836,-1.176462
2928,-1.032058,-0.218968,-1.176462


Notice that the resulting `DataFrame` contains negative values. This makes sense because standardizing makes the mean of every variable equal to 0. If the mean is 0, then some values must be negative.

The above command is deceptively simple. We actually subtracted a `DataFrame` by a `Series`, then divided the resulting `DataFrame` by another `Series`. We relied on `pandas` to broadcast each `Series` over the right dimension of the `DataFrame`. To be more explicit about the broadcasting, we could have also used the `.sub()` and `.divide()` methods (instead of `-` and `/`) and been explicit about the axis:

In [24]:
df_housing_st = (df_housing_quant.
                  sub(df_housing_quant.mean(), axis=1).
                  divide(df_housing_quant.std(), axis=1))
df_housing_st

,Bedroom AbvGr,Gr Liv Area,Bathrooms
0,0.176064,0.309212,-1.176462
1,-1.032058,-1.194223,-1.176462
2,0.176064,-0.337661,-0.398702
3,0.176064,1.207317,1.156819
4,0.176064,0.255801,1.156819
...,...,...,...
2925,0.176064,-0.982555,-1.176462
2926,-1.032058,-1.182354,-1.176462
2927,0.176064,-1.047836,-1.176462
2928,-1.032058,-0.218968,-1.176462


Now let's recalculate the distances using this standardized data and see if our conclusions change.

In [25]:
# Distance between two points connected by red line
x = df_housing_st.loc[2927]
x1 = df_housing_st.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

1.4651211129695825

In [26]:
# Distance between two points connected by orange line
x = df_housing_st.loc[2498]
x1 = df_housing_st.loc[290]

np.sqrt(((x - x1) ** 2).sum())

3.9440754446060033

So, if we first standardize the data, then the pair of observations connected by the red line are more similar than the pair connected by the orange line, which matches our intuition. It is (almost) always a good idea to scale the variables before calculating distances.

## The Scikit-Learn API

Scikit-Learn is a machine learning library in Python that we will use extensively in Part II of this book. Since scaling data and calculating distances are essential tasks in machine learning, scikit-learn has built-in functions for carrying out these common tasks.

To scale a variable in scikit-learn, there are three steps:

1. First, we declare the scaler that we want to use.
2. Next, we "fit" the scaler to data. For example, in the case of standardization, this simply calculates and stores the mean and standard deviation to use for standardization.
3. Finally, we transform the data. This actually applies the scaling to the data.

To standardize data, we use the `StandardScaler`. There is also a `MinMaxScaler` for min-max scaling and a `Normalizer` for normalization (but scikit-learn's `Normalizer` normalizes the rows to be length 1, rather than the columns). See [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) for a complete list of scalers and other preprocessing functions.

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_housing_quant)
df_housing_st = scaler.transform(df_housing_quant)

df_housing_st

array([[ 0.17609421,  0.30926506, -1.17666295],
       [-1.03223376, -1.19442705, -1.17666295],
       [ 0.17609421, -0.33771825, -0.3987698 ],
       ...,
       [ 0.17609421, -1.04801492, -1.17666295],
       [-1.03223376, -0.21900572, -1.17666295],
       [ 0.17609421,  0.9898836 ,  1.1570165 ]])

Notice that scikit-learn returns the standardized data as a plain `numpy` array, rather than a `pandas` `DataFrame`. This is one disadvantage of using scikit-learn; we lose the column names, the row index, and all of the other metadata that `DataFrame`s contain.

You might wonder why scikit-learn divides scaling into three separate steps. For example, why is it necessary to separate the fitting step from the transformation step? The reason is that a scaler can be fit to one data set and then used to transform many different data sets, not just the original data set to which it was fit. Since the scaler is fit only once, this guarantees that all subsequent data sets will be scaled in exactly the same way (i.e., with respect to the same mean and standard deviation if using the `StandardScaler`).

Scikit-Learn also has built-in functions for calculating distances. For example, to calculate all pairwise distances between observations (2927, 2498) and (2928, 290), we can use the `euclidean_distances` function. There are also other distance metrics available, such as `manhattan_distances`.

In [28]:
from sklearn.metrics.pairwise import euclidean_distances

x = df_housing_st[[2927, 2498], :]
x1 = df_housing_st[[2928, 290], :]

euclidean_distances(x, x1)

array([[1.4653712 , 5.81988338],
       [3.17266983, 3.94474867]])

The upper left entry of this matrix represents the distance between observations 2927 and 2928, while the lower right entry represents the distance between observations 2498 and 290. Check that they match the distances we calculated earlier using `pandas`.

## Exercises

1\. Instead of standardizing the three variables from the Ames housing data set, normalize them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [29]:
from sklearn.preprocessing import Normalizer

scaler = StandardScaler()
scaler.fit(df_housing_quant)
df_housing_st = scaler.transform(df_housing_quant)

2\. Instead of standardizing the three variables from the Ames housing data set, apply a min-max scaling to them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler =

SyntaxError: ignored

3\. Suppose that you really like house 0 in the data set, but it is too expensive. Find cheaper homes that are similar to it, by calculating distances after encoding categorical variables as dummy variables. Be sure to actually look at the profiles of the homes that your algorithm picked out as most similar. Do they make sense?

Try different distance metrics and different scaling methods. How sensitive are your results to these choices?

_Think:_ If the goal is to find a "good deal" on a similar house, should sale price be included as a variable in your distance metric? 

_Hint:_ There are too many variables in the data set. Do not attempt to call `pd.get_dummies()` on the entire `DataFrame`! You will want to pare down the number of variables, but be sure to include a mixture of categorical and quantitative variables. Refer to the [data documentation](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) for information about the variables.

In [40]:
df_housing.iloc[0]
df_hosing_vars = df_housing[['Lot Frontage', 'Lot Area', 'Bldg Type', 'SalePrice']]

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
vals = scaler.fit_transform(df_hosing_vars[['Lot Frontage', 'Lot Area']])
vals

array([[ 3.07250559,  2.74438073],
       [ 0.46126531,  0.18709726],
       [ 0.50407253,  0.5228137 ],
       ...,
       [-0.3092646 ,  0.03719892],
       [ 0.33284366, -0.01750572],
       [ 0.20442201, -0.06611797]])

In [47]:
df_housing[['Lot Frontage Scaled', 'Lot Area Scaled']] = vals

In [53]:
df_housing_with_dummies = pd.get_dummies(df_housing[['Lot Area Scaled', 'Lot Frontage Scaled','Bldg Type']], columns = ['Bldg Type'])
df_housing_with_dummies.head()

,Lot Area Scaled,Lot Frontage Scaled,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE
0,2.744381,3.072506,1,0,0,0,0
1,0.187097,0.461265,1,0,0,0,0
2,0.522814,0.504073,1,0,0,0,0
3,0.128458,1.017759,1,0,0,0,0
4,0.467348,0.204422,1,0,0,0,0


In [61]:
distance = euclidean_distances(df_housing_with_dummies.fillna(0).loc[[0]], df_housing_with_dummies.fillna(0))

In [63]:
distance.shape

(1, 2930)

In [64]:
df_housing['distance'] = distance[0]

In [65]:
df_housing['distance'].describe()

count    2930.000000
mean        4.259790
std         1.001959
min         0.000000
25%         3.783545
50%         4.210995
75%         4.663484
max        23.290738
Name: distance, dtype: float64

In [69]:
df_housing[(df_housing['distance']  < 1) & df_housing['SalePrice'] < df_housing.loc[0]['SalePrice']].sort_values('distance')

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Lot Frontage Scaled,Lot Area Scaled,distance
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,NaN,0,5,2010,WD,Normal,215000,3.072506,2.744381,0.000000
2903,2904,923125030,20,A (agr),125.0,31250,Pave,NaN,Reg,Lvl,...,NaN,0,5,2006,WD,Normal,81500,2.387590,2.678380,0.688088
2298,2299,923251160,20,RL,124.0,27697,Pave,NaN,Reg,Lvl,...,NaN,0,11,2007,COD,Abnorml,80000,2.344783,2.227415,0.892655
2180,2181,908154195,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,...,Elev,17000,10,2007,New,Partial,183850,2.516012,3.698856,1.104857
2445,2446,528320060,60,RL,118.0,35760,Pave,NaN,IR1,Lvl,...,NaN,0,7,2006,WD,Normal,625000,2.087940,3.250811,1.107178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,1499,908154235,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,...,NaN,0,1,2008,New,Partial,160000,10.435347,6.820829,8.415988
2071,2072,905301050,20,RL,NaN,115149,Pave,NaN,IR2,Low,...,NaN,0,6,2007,WD,Normal,302000,NaN,13.327254,11.019869
2115,2116,906426060,50,RL,NaN,159000,Pave,NaN,IR2,Low,...,Shed,500,6,2007,WD,Normal,277000,NaN,18.893039,16.438354
1570,1571,916125425,190,RL,NaN,164660,Grvl,NaN,IR1,HLS,...,Shed,700,8,2008,WD,Normal,228950,NaN,19.611435,17.202843
